In [1]:
import numpy as np
import pandas as pd
import sys

from sklearn.pipeline import Pipeline

from methods import binning, normalise, smoothing, baseline, FeaExtraction, Classifier, Scattering, utils


In [2]:
data = pd.read_hdf('/home/sgcwhitl/Preprocess_Opt/Example_Job/data.hdf')

In [3]:
data.shape

(299607, 169)

In [4]:
pipe = Pipeline([('Kohler_test', Scattering.Kohler(n_jobs=1))])

In [5]:
samp = data.sample(1000)

transformed = pipe.fit_transform(data.sample(1000))

TypeError: Remote functions cannot be called directly. Instead of running 'methods.Scattering.Kohler_fit()', try 'methods.Scattering.Kohler_fit.remote()'.

In [2]:
transformed.mean().plot()

NameError: name 'transformed' is not defined